# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-aux-francais,-49.3500,70.2167,4.13,88,100,15.18,TF,1727399253
1,1,waitangi,-43.9535,-176.5597,8.23,97,100,4.02,NZ,1727399254
2,2,adamstown,-25.0660,-130.1015,21.66,88,73,4.93,PN,1727399256
3,3,kargasok,59.0556,80.8572,0.23,97,100,3.59,RU,1727399257
4,4,wailua homesteads,22.0669,-159.3780,28.44,72,100,5.14,US,1727399258


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800, 
    frame_height = 600, 
    size = "Humidity",
    scale = 1, 
    color = "City"
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [15]:
# Narrow down cities that fit criteria and drop any results with null values
my_ideal_cities = city_data_df.loc[(city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 27) & 
                                (city_data_df["Wind Speed"] < 4.5)]

# Drop any rows with null values
my_ideal_cities = my_ideal_cities.dropna()
# Display sample data
my_ideal_cities

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,ribeira brava,32.6500,-17.0667,22.35,79,0,1.57,PT,1727399267
17,17,puerto armuelles,8.2833,-82.8667,25.61,88,100,1.79,PA,1727399273
31,31,morehead city,34.7229,-76.7260,25.03,92,100,1.79,US,1727399289
35,35,bethel,41.3712,-73.4140,22.05,90,100,2.06,US,1727399293
41,41,rabkavi,16.4667,75.1000,21.29,93,65,3.23,IN,1727399300
...,...,...,...,...,...,...,...,...,...,...
564,564,bardai,11.6500,7.2333,23.20,79,100,3.02,NG,1727399918
568,568,woodward,36.4337,-99.3904,23.31,46,0,2.57,US,1727399925
569,569,sihora,23.4833,80.1167,23.70,96,100,3.83,IN,1727399926
574,574,machico,32.7000,-16.7667,22.35,75,20,2.57,PT,1727399933


### Step 3: Create a new DataFrame called `hotel_df`.

In [16]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_cities[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
12,ribeira brava,PT,32.6500,-17.0667,79,
17,puerto armuelles,PA,8.2833,-82.8667,88,
31,morehead city,US,34.7229,-76.7260,92,
35,bethel,US,41.3712,-73.4140,90,
41,rabkavi,IN,16.4667,75.1000,93,
...,...,...,...,...,...,...
564,bardai,NG,11.6500,7.2333,79,
568,woodward,US,36.4337,-99.3904,46,
569,sihora,IN,23.4833,80.1167,96,
574,machico,PT,32.7000,-16.7667,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [17]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
   "limit":1,
    "apiKey":geoapify_key,
    "categories":categories
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    my_row_lat = row["Lat"]
    my_row_lon = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    filters = f"circle:{my_row_lon},{my_row_lat},{radius}"
    bias = f"proximity:{my_row_lon},{my_row_lat}"
    params["filter"] = filters
    params["bias"] = bias

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ribeira brava - nearest hotel: Cheerfulway Bravamar Hotel
puerto armuelles - nearest hotel: Coco del Mar
morehead city - nearest hotel: Bask Hotel Big Rock Landing, Trademark Collection by Wyndham
bethel - nearest hotel: Hampton Inn Danbury
rabkavi - nearest hotel: Hotel Royal Residency
ha'il - nearest hotel: فندق كلاسيك
rock springs - nearest hotel: Sands Inn
avarua - nearest hotel: Paradise Inn
dingli - nearest hotel: Maple farm B&B
holualoa - nearest hotel: Kona Hotel
greenville - nearest hotel: Quality Inn
crane - nearest hotel: No hotel found
san patricio - nearest hotel: No hotel found
batticaloa - nearest hotel: Lake View Hotel
sakaiminato - nearest hotel: かぐら旅館
sao tome - nearest hotel: Hotel Baia
famagusta - nearest hotel: Portofino Hotel
nova sintra - nearest hotel: Residência Ka Dencho
kodigenahalli - nearest hotel: No hotel found
flying fish cove - nearest hotel: Christmas Island Lodge
portland - nearest hotel: the Hoxton
city of balikpapan - nearest h

,City,Country,Lat,Lng,Humidity,Hotel Name
12,ribeira brava,PT,32.6500,-17.0667,79,Cheerfulway Bravamar Hotel
17,puerto armuelles,PA,8.2833,-82.8667,88,Coco del Mar
31,morehead city,US,34.7229,-76.7260,92,"Bask Hotel Big Rock Landing, Trademark Collect..."
35,bethel,US,41.3712,-73.4140,90,Hampton Inn Danbury
41,rabkavi,IN,16.4667,75.1000,93,Hotel Royal Residency
...,...,...,...,...,...,...
564,bardai,NG,11.6500,7.2333,79,No hotel found
568,woodward,US,36.4337,-99.3904,46,Sands Inn
569,sihora,IN,23.4833,80.1167,96,No hotel found
574,machico,PT,32.7000,-16.7667,75,Residence Inn


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [20]:
%%capture --no-display

# Configure the map plot
hotel_plot = hotel_df.hvplot.points( 
    "Lng",
    "Lat",   
    geo= True,      
    tiles = "OSM",  
    frame_width = 700, 
    frame_height = 500, 
    size = "Humidity",
    scale = 1, 
    color = "City",
)

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)